In [1]:
## Initial setup
import qiskit
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler
import os
from dotenv import load_dotenv

load_dotenv()
API_TOKEN = os.getenv("API_TOKEN")
service = QiskitRuntimeService(channel="ibm_quantum", token=API_TOKEN)
backend = service.backend(name="ibm_rensselaer")

In [2]:
# Create circuit
qc = qiskit.QuantumCircuit(4)
qc.initialize('0000')

# transform input of 0s to superposition
qc.h(0)

qc.h(1)

qc.x(2)
qc.h(2)

qc.x(3)
qc.h(3)

# apply Uf
qc.x(0)

qc.ccx(0, 1, 3)

qc.x(0)
qc.x(1)

qc.ccx(0, 1, 3)

qc.cx(0, 1)

qc.ccx(1, 2, 3)

qc.cx(0, 1)

qc.x(1)

# finally measure
qc.measure_all()


qc.draw()

┌──────────────────────┐┌───┐┌───┐     ┌───┐                         »
   q_0: ┤0                     ├┤ H ├┤ X ├──■──┤ X ├──■────■─────────■───────»
        │                      │├───┤└───┘  │  ├───┤  │  ┌─┴─┐     ┌─┴─┐┌───┐»
   q_1: ┤1                     ├┤ H ├───────■──┤ X ├──■──┤ X ├──■──┤ X ├┤ X ├»
        │  Initialize(0,0,0,0) │├───┤┌───┐  │  └───┘  │  └───┘  │  └───┘└───┘»
   q_2: ┤2                     ├┤ X ├┤ H ├──┼─────────┼─────────■────────────»
        │                      │├───┤├───┤┌─┴─┐     ┌─┴─┐     ┌─┴─┐          »
   q_3: ┤3                     ├┤ X ├┤ H ├┤ X ├─────┤ X ├─────┤ X ├──────────»
        └──────────────────────┘└───┘└───┘└───┘     └───┘     └───┘          »
meas: 4/═════════════════════════════════════════════════════════════════════»
                                                                             »
«         ░ ┌─┐         
«   q_0: ─░─┤M├─────────
«         ░ └╥┘┌─┐      
«   q_1: ─░──╫─┤M├──────
«         ░  ║ └╥┘┌─┐   
«   q_2: ─░──╫──╫─┤M├───
«         ░  ║  ║ └╥┘┌─┐
«   q_3: ─░──╫──╫──╫─┤M├
«         ░  ║  ║  ║ └╥┘
«meas: 4/════╩══╩══╩══╩═
«            0  1  2  3

In [3]:
# Transpile and execute
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import EstimatorV2 as Estimator

pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
qc_transpiled = pm.run(qc)

sampler = Sampler(backend)
job = sampler.run([qc_transpiled], shots=10**4)
result = job.result()[0]

In [4]:
# Display results
counts = result.data.meas.get_counts()
print("z x3 x2 x1 | frequency")
for i in range(16):
    binary = format(i, '04b')
    print(binary, "      |", counts.get(binary, 0))

z x3 x2 x1 | frequency
0000       | 1052
0001       | 586
0010       | 379
0011       | 565
0100       | 408
0101       | 1145
0110       | 478
0111       | 323
1000       | 726
1001       | 683
1010       | 210
1011       | 306
1100       | 1235
1101       | 941
1110       | 329
1111       | 634


In [5]:
# Simulation comparison
from qiskit_aer import AerSimulator
simulator = AerSimulator()
pm = generate_preset_pass_manager(optimization_level=1, backend=simulator)
qc_transpiled = pm.run(qc)

simulation = simulator.run(qc_transpiled, shots=10**4)
simulation_counts = simulation.result().get_counts()

print("z x3 x2 x1 | frequency")
for i in range(16):
    binary = format(i, '04b')
    print(binary, "      |", simulation_counts.get(binary, 0))

z x3 x2 x1 | frequency
0000       | 649
0001       | 595
0010       | 610
0011       | 609
0100       | 597
0101       | 611
0110       | 613
0111       | 614
1000       | 649
1001       | 675
1010       | 637
1011       | 632
1100       | 627
1101       | 633
1110       | 599
1111       | 650
